In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [3]:

def find_output_files(rank, data_dir):
    """Find output files from Woltka classify.
    
    Parameters
    ----------
    rank : str
        Taxonomic rank.
    data_dir : str
        Path to directory containing output files.
    
    Returns
    -------
    list
        List of paths to output files.
    """
    output_files = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if f"{rank}.tsv" == file:
                output_files.append(os.path.join(root, file))
    return output_files

In [4]:
def rel_abundance(df, output, rank, plot=False):
    # df.drop("FeatureID", axis=1, inplace=True)
    # pct = df[["Count"]].apply(lambda x: x / x.sum(), axis=0)
    df["RA"] = df["Count"] / df["Count"].sum()
    df.drop("Count", axis=1, inplace=True)
    df = df.sort_values(by="RA", ascending=False)

    # Remove any [ and ] from the index.
    df.index = df.index.str.replace("[", "", regex=False)
    df.index = df.index.str.replace("]", "", regex=False)

    df = df.where(df > 5e-5).dropna()

    final_df = df[["RA", "TAX_ID"]]
    final_df.to_csv(output, sep=",")

    if plot:
        final_df.T.plot.bar(figsize=(10,10), xlabel="{rank} Name", ylabel="Fraction", title=f"{rank} Relative Abundance above 0.005%").legend(loc='center left', bbox_to_anchor=(1.0, 0.5), title=f"{rank}")
        plt.savefig(output + ".png", bbox_inches='tight')

In [5]:
# Find output files.
def find_and_save(input_data: str, rank: str, output_dir: str):
    """Find and save output files from Woltka classify.
    
    Parameters
    ----------
    input_data : str
        Path to directory containing output files.
    rank : str
        Taxonomic rank.
    output_dir : str
        Path to directory to save output files.
    """
    output_files = find_output_files(rank, input_data)
    for file in output_files:
        print(file)
        df = pd.read_csv(file, sep="\t", names=["TAX_ID", "Count", "Species"], header=0, index_col=2)

        sampleID = (os.path.dirname(file).split("/")[-1]).split("_")[0]
        output_path = os.path.join(output_dir, f"{sampleID}_{rank}_relabund_annotated.csv")
        rel_abundance(df, output_path, rank)

# Main

In [6]:
data_dir = "/Volumes/TBHD_share/valencia/pipelines/bmock12/woltka/classify/results"
output_dir = "../../pipelines/bmock12/woltka"

find_and_save(input_data=data_dir, rank="genus", output_dir=output_dir)
find_and_save(input_data=data_dir, rank="species", output_dir=output_dir)

/Volumes/TBHD_share/valencia/pipelines/bmock12/woltka/classify/results/genus.tsv
/Volumes/TBHD_share/valencia/pipelines/bmock12/woltka/classify/results/species.tsv
